In [2]:
!pip install autogluon

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import math
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_absolute_error

 ### 데이터 불러오기

In [5]:
winter = pd.read_csv("/content/drive/MyDrive/날씨 빅데이터 공모전/data/winter_train.csv")
winter_test = pd.read_csv("/content/drive/MyDrive/날씨 빅데이터 공모전/data/winter_test.csv")

### mmddhh 칼럼 월 일 시간으로 나누기

In [6]:
winter["hour"] = winter["mmddhh"] % 100
winter["day"] = (np.trunc((winter["mmddhh"] % 10000) / 100))
winter["month"] = np.trunc(winter["mmddhh"] / 10000)

winter_test["hour"] = winter_test["mmddhh"] % 100
winter_test["day"] = (np.trunc((winter_test["mmddhh"] % 10000) / 100))
winter_test["month"] = np.trunc(winter_test["mmddhh"] / 10000)

### 지면온도 예측에 대한 RN(누적강수량)의 일부 값을 변경

In [7]:
winter = winter.replace({"1시간 누적 강수량(mm)" : [1.8, 3.3, 5.3, 19.5, 30.3, 623.5]}, -99.9)

winter_test = winter_test.replace({"1시간 누적 강수량(mm)" : [1.8, 3.3, 5.3, 19.5, 30.3, 623.5]}, -99.9)

### 1시간 누적 일사량과 1시간 누적 일조량 칼럼 수정

In [8]:
winter_11 = [0, 1, 2, 3, 4, 5, 6, 7, 19, 20, 21, 22, 23]
winter_12 = [0, 1, 2, 3, 4, 5, 6, 7, 19, 20, 21, 22, 23]
winter_1 = [0, 1, 2, 3, 4, 5, 6, 7, 19, 20, 21, 22, 23]

for i in winter_11 :
    winter.loc[(winter["1시간 누적 일사량(MJ)"] == -99.9) & (winter["hour"] == i) & (winter["월"] == 11), "1시간 누적 일사량(MJ)"] = 0
    winter.loc[(winter["1시간 누적 일조량(초)"] == -99.9) & (winter["hour"] == i) & (winter["월"] == 11), "1시간 누적 일조량(초)"] = 0

for i in winter_12 :
    winter.loc[(winter["1시간 누적 일사량(MJ)"] == -99.9) & (winter["hour"] == i) & (winter["월"] == 12), "1시간 누적 일사량(MJ)"] = 0
    winter.loc[(winter["1시간 누적 일조량(초)"] == -99.9) & (winter["hour"] == i) & (winter["월"] == 12), "1시간 누적 일조량(초)"] = 0

for i in winter_1 :
    winter.loc[(winter["1시간 누적 일사량(MJ)"] == -99.9) & (winter["hour"] == i) & (winter["월"] == 1), "1시간 누적 일사량(MJ)"] = 0
    winter.loc[(winter["1시간 누적 일조량(초)"] == -99.9) & (winter["hour"] == i) & (winter["월"] == 1), "1시간 누적 일조량(초)"] = 0


for i in winter_11 :
    winter_test.loc[(winter_test["1시간 누적 일사량(MJ)"] == -99.9) & (winter_test["hour"] == i) & (winter_test["월"] == 11), "1시간 누적 일사량(MJ)"] = 0
    winter_test.loc[(winter_test["1시간 누적 일조량(초)"] == -99.9) & (winter_test["hour"] == i) & (winter_test["월"] == 11), "1시간 누적 일조량(초)"] = 0

for i in winter_12 :
    winter_test.loc[(winter_test["1시간 누적 일사량(MJ)"] == -99.9) & (winter_test["hour"] == i) & (winter_test["월"] == 12), "1시간 누적 일사량(MJ)"] = 0
    winter_test.loc[(winter_test["1시간 누적 일조량(초)"] == -99.9) & (winter_test["hour"] == i) & (winter_test["월"] == 12), "1시간 누적 일조량(초)"] = 0

for i in winter_1 :
    winter_test.loc[(winter_test["1시간 누적 일사량(MJ)"] == -99.9) & (winter_test["hour"] == i) & (winter_test["월"] == 1), "1시간 누적 일사량(MJ)"] = 0
    winter_test.loc[(winter_test["1시간 누적 일조량(초)"] == -99.9) & (winter_test["hour"] == i) & (winter_test["월"] == 1), "1시간 누적 일조량(초)"] = 0


winter.loc[(winter["1시간 누적 일사량(MJ)"] == -99.9) & (winter["1시간 누적 일조량(초)"] == 0), "1시간 누적 일사량(MJ)"] = 0
winter.loc[(winter["1시간 누적 일사량(MJ)"] == 0) & (winter["1시간 누적 일조량(초)"] == -99.9), "1시간 누적 일조량(초)"] = 0

winter_test.loc[(winter_test["1시간 누적 일사량(MJ)"] == -99.9) & (winter_test["1시간 누적 일조량(초)"] == 0), "1시간 누적 일사량(MJ)"] = 0
winter_test.loc[(winter_test["1시간 누적 일사량(MJ)"] == 0) & (winter_test["1시간 누적 일조량(초)"] == -99.9), "1시간 누적 일조량(초)"] = 0


### 결측치 처리

In [9]:
winter = winter.replace([-99, -99.9, -999], np.nan)

winter_test = winter_test.replace([-99, -99.9, -999], np.nan)

### 사용하지 않는 칼럼 제거

In [10]:
winter = winter.drop(["mmddhh", "적설 깊이(cm)", "월"], axis = 1)

winter_test = winter_test.drop(["mmddhh", "적설 깊이(cm)", "월"], axis = 1)

### 1시간 누적 강수유무와 1시간 누적 강수량 칼럼 수정

In [11]:
winter.loc[(winter["1시간 누적 강수유무(분)"]== 0) & (winter["1시간 누적 강수량(mm)"].isnull()), "1시간 누적 강수량(mm)"] = 0
winter.loc[winter["1시간 누적 강수량(mm)"].isnull(), "1시간 누적 강수량(mm)"] = 0
winter = winter.drop("1시간 누적 강수유무(분)", axis = 1)

winter_test.loc[(winter_test["1시간 누적 강수유무(분)"]== 0) & (winter_test["1시간 누적 강수량(mm)"].isnull()), "1시간 누적 강수량(mm)"] = 0
winter_test.loc[winter_test["1시간 누적 강수량(mm)"].isnull(), "1시간 누적 강수량(mm)"] = 0
winter_test = winter_test.drop("1시간 누적 강수유무(분)", axis = 1)

### 인덱스 초기화

In [12]:
winter = winter.reset_index(drop=True)

### 라벨 인코딩 수행

In [13]:
cha = ["지점", "연도", "현천"]

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for i in cha:
    winter[i] = le.fit_transform(winter[i])

    for label in np.unique(winter_test[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)

    winter_test[i] = le.transform(winter_test[i])

<ipython-input-13-cc39c1289101>:11: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if label not in le.classes_:



## 파생변수 추가

### 1. 기온 변화량

In [14]:
# 파생변수 추가
winter = winter.reset_index(drop = True)
winter_test = winter_test.reset_index(drop=True)

winter["기온변화량"] = -(winter["1시간 평균 기온(C)"] - winter["1시간 평균 기온(C)"][1:].reset_index(drop = True))
winter["기온변화량"] = winter["기온변화량"].shift(1)

winter_test["기온변화량"] = -(winter_test["1시간 평균 기온(C)"] - winter_test["1시간 평균 기온(C)"][1:].reset_index(drop = True))
winter_test["기온변화량"] = winter_test["기온변화량"].shift(1)


### 2. 상대습도 변화량

In [15]:
winter["상대습도변화량"] = -(winter["1시간 평균 상대습도(%)"] - winter["1시간 평균 상대습도(%)"][1:].reset_index(drop = True))
winter["상대습도변화량"] = winter["상대습도변화량"].shift(1)

winter_test["상대습도변화량"] = -(winter_test["1시간 평균 상대습도(%)"] - winter_test["1시간 평균 상대습도(%)"][1:].reset_index(drop = True))
winter_test["상대습도변화량"] = winter_test["상대습도변화량"].shift(1)


### 3. 평균 기온 * 평균 상대습도

In [16]:
winter["평균기온*평균상대습도"] = winter["1시간 평균 기온(C)"] * winter["1시간 평균 상대습도(%)"]

winter_test["평균기온*평균상대습도"] = winter_test["1시간 평균 기온(C)"] * winter_test["1시간 평균 상대습도(%)"]


### 4. 1시간 누적 일사량 * 1시간 누적 일조량

In [17]:
winter["1시간 누적 일사량 * 1시간 누적 일조량"] = winter["1시간 누적 일사량(MJ)"] * winter["1시간 누적 일조량(초)"]

winter_test["1시간 누적 일사량 * 1시간 누적 일조량"] = winter_test["1시간 누적 일사량(MJ)"] * winter_test["1시간 누적 일조량(초)"]

### 2개 이상 연속되는 결측치 찾기 및 제거

In [18]:
lists = ["1시간 평균 기온(C)", "1시간 평균 이슬점온도(C)", "1시간 평균 상대습도(%)", "1시간 평균풍속(m/s)" , "1시간 평균 지면온도(C)", "1시간 누적 일사량(MJ)", "1시간 누적 일조량(초)"
,"기온변화량" ,"상대습도변화량", "평균기온*평균상대습도", "1시간 누적 일사량 * 1시간 누적 일조량"]
for i in lists:
    ans = winter[winter[i].isnull()].index
    cons = []
    for j in range(len(ans)-1) :
        if ans[j+1] - ans[j] == 1 : # i+1번째 숫자와 i번째 숫자의 차이가 1이라면(연속이라면)
            cons.extend([ans[j],ans[j+1]]) #i+1번째 숫자와 i번째 숫자를 결과 출력할 리스트에 추가
    delete = list(set(cons)) #위 알고리즘으로 하면 만일 숫자가 23,24,25라면 23, 24, 24, 25로 저장되기 때문에 중복되는 숫자를 제거

    winter = winter.drop(winter.index[delete])
    #인덱스를 초기화
    winter = winter.reset_index(drop=True)

In [19]:
winter

,지점,연도,1시간 평균 기온(C),1시간 평균 이슬점온도(C),1시간 평균 상대습도(%),1시간 평균풍속(m/s),1시간 누적 강수량(mm),현천,1시간 평균 지면온도(C),1시간 누적 일사량(MJ),1시간 누적 일조량(초),hour,day,month,기온변화량,상대습도변화량,평균기온*평균상대습도,1시간 누적 일사량 * 1시간 누적 일조량
0,0,0,6.8,4.8,87.5,0.2,0.0,0,5.9,0.0,0.0,0,1.0,11.0,NaN,NaN,595.00,0.0
1,0,0,6.6,5.1,90.1,0.4,0.0,3,5.7,0.0,0.0,1,1.0,11.0,-0.2,2.6,594.66,0.0
2,0,0,6.6,5.1,90.1,0.3,0.0,3,6.0,0.0,0.0,2,1.0,11.0,-0.0,-0.0,594.66,0.0
3,0,0,6.4,5.4,93.2,0.3,0.0,3,5.5,0.0,0.0,3,1.0,11.0,-0.2,3.1,596.48,0.0
4,0,0,6.3,5.4,94.1,0.3,0.0,3,5.7,0.0,0.0,4,1.0,11.0,-0.1,0.9,592.83,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109951,9,5,5.7,-7.2,39.2,3.9,0.0,0,3.6,0.0,0.0,19,31.0,1.0,-1.4,8.3,223.44,0.0
109952,9,5,5.2,-8.0,38.1,4.9,0.0,0,2.7,0.0,0.0,20,31.0,1.0,-0.5,-1.1,198.12,0.0
109953,9,5,4.6,-7.9,39.8,6.6,0.0,0,2.2,0.0,0.0,21,31.0,1.0,-0.6,1.7,183.08,0.0
109954,9,5,3.6,-6.6,47.4,7.1,0.0,0,1.6,0.0,0.0,22,31.0,1.0,-1.0,7.6,170.64,0.0


### 결측치 채우기 : 선형 보간

In [20]:
winter = winter.interpolate(method = 'linear')

winter_test = winter_test.interpolate(method = 'linear')

### autogluon 적용

In [21]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [22]:
# 0~2년도의 데이터를 train, 3년도의 데이터를 validation, 4년도의 데이터를 test로 하여 autogluon 적용

auto_winter_train = winter.loc[(winter["연도"] != 3) & (winter["연도"] != 4)]
auto_winter_val = winter.loc[winter["연도"] == 3]
auto_winter_test = winter.loc[winter["연도"] == 4]

In [23]:
predictor = TabularPredictor(label="1시간 평균 지면온도(C)", eval_metric = "mae").fit(auto_winter_train, tuning_data = auto_winter_val)

No path specified. Models will be saved in: "AutogluonModels/ag-20230705_025214/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230705_025214/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Apr 29 09:15:28 UTC 2023
Disk Space Avail:   83.44 GB / 115.66 GB (72.1%)
Train Data Rows:    66007
Train Data Columns: 17
Tuning Data Rows:    21983
Tuning Data Columns: 17
Label Column: 1시간 평균 지면온도(C)
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and label-values can't be converted to int).
	Label info (max, min, mean, stddev): (37.7, -19.9, 3.19116, 7.13448)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preproce

In [24]:
predictor.leaderboard(auto_winter_test, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMLarge,-1.387865,-1.445910,0.384251,0.282992,3.886678,0.384251,0.282992,3.886678,1,True,11
1,WeightedEnsemble_L2,-1.393319,-1.405419,18.714758,32.807170,245.879325,0.012405,0.006411,0.737765,2,True,12
2,XGBoost,-1.427350,-1.440082,0.156780,0.116294,4.513826,0.156780,0.116294,4.513826,1,True,9
3,CatBoost,-1.428795,-1.443064,0.170774,0.011700,4.756343,0.170774,0.011700,4.756343,1,True,6
4,LightGBM,-1.430785,-1.459961,0.203804,0.229568,4.002533,0.203804,0.229568,4.002533,1,True,4
5,ExtraTreesMSE,-1.445866,-1.484559,3.230205,1.185450,69.240599,3.230205,1.185450,69.240599,1,True,7
6,LightGBMXT,-1.446548,-1.486957,0.261106,0.292144,4.600216,0.261106,0.292144,4.600216,1,True,3
7,RandomForestMSE,-1.496259,-1.549784,4.533864,2.349283,214.753864,4.533864,2.349283,214.753864,1,True,5
8,NeuralNetFastAI,-1.552427,-1.516100,0.732620,0.585192,126.218709,0.732620,0.585192,126.218709,1,True,8
9,NeuralNetTorch,-1.593246,-1.592288,0.191976,0.110658,98.142617,0.191976,0.110658,98.142617,1,True,10


### winter_test에 적용

In [25]:
winter_pred = predictor.predict(winter_test)

In [26]:
winter_pred

0       10.504467
1       10.076106
2       10.180672
3        9.949140
4        9.896210
          ...    
6619     5.767205
6620     5.155311
6621     4.629985
6622     4.836035
6623     4.772182
Name: 1시간 평균 지면온도(C), Length: 6624, dtype: float32

In [27]:
import pandas as pd

# 파일명
file_name = '/content/drive/MyDrive/날씨 빅데이터 공모전/data/subminssionUser.xlsx'

writer = pd.ExcelWriter(file_name, mode='a', engine='openpyxl', if_sheet_exists='overlay')
submission_winter = pd.read_excel(file_name, sheet_name= "WINTER")

# Daraframe형식으로 엑셀 파일 읽기
submission_winter = pd.read_excel(file_name, sheet_name= "WINTER")

#제출 파일에 빠진 날짜 찾아서 테스트에서도 빼버리기
submission_winter["TS"] = winter_pred

submission_winter.to_excel(
    writer,
    sheet_name='WINTER',
    index=False,
    # header = None
    )
writer.save()
writer.close()

<ipython-input-27-8ff1556cc504>:21: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
